# Simulators and datasets

In `lampe`, a simulator can be any Python callable that takes (a vector of) parameters $\theta$ as input, in the form of a NumPy array or a PyTorch tensor, and returns a stochastic observation $x$ as output. As such, a simulator implicitly defines a likelihood distribution $p(x | \theta)$. The prior distribution $p(\theta)$ is restricted to be a [PyTorch distribution](https://pytorch.org/docs/stable/distributions.html) implementing the `sample` and `log_prob` methods. Together, the prior and the simulator form a joint distribution $p(\theta, x) = p(\theta) p(x | \theta)$ from which parameters-observation pairs $(\theta, x)$ can be drawn.

This notebook walks you through setting up a prior and a simulator, sampling pairs from the joint, saving pairs on disk and, finally, loading pairs from disk.

In [1]:
import lampe
import torch

from itertools import islice
from tqdm import tqdm

## Prior

In Bayesian inference problems, it is quite common to have a uniform prior distribution $p(\theta)$. The [`lampe.distributions`](https://francois-rozet.github.io/lampe/api/distributions/index.html) module provides helpers to build distributions. Notably, the [`BoxUniform`](https://francois-rozet.github.io/lampe/api/distributions/index.html#lampe.distributions.BoxUniform) class creates a multivariate uniform distribution between a lower bound and an upper bound.

For instance, let's say $\theta$ is uniform over the domain $[-1, 1]^3$.

In [2]:
LOWER = -torch.ones(3)
UPPER = torch.ones(3)

prior = lampe.distributions.BoxUniform(LOWER, UPPER)
prior

BoxUniform(low: torch.Size([3]), high: torch.Size([3]))

Using the `sample` method, parameters `\theta` can be sampled from the newly created prior distribution. The log-density can also be evaluated for given parameters using the `log_prob` method.

In [3]:
theta = prior.sample()
theta

tensor([-0.7064,  0.0166, -0.3484])

In [4]:
prior.log_prob(theta)

tensor(-2.0794)

## Simulator

As mentioned previously, the only constraints for the simulator are that the vector of parameters $\theta$ and the observation $x$ must either be NumPy arrays or PyTorch tensors. The callable is otherwise unconstrained, which allows for a large variety of simulators, including already existing ones and ones implemented in other programming languages than Python. A few examples of simulators compatible with `lampe` are provided at https://github.com/francois-rozet/sbi-benchmarks.

Let's consider the following generative process as our simulator:

$$ x = \begin{pmatrix} \theta_1 + \theta_2 \times \theta_3 \\ \theta_1 \times \theta_2 + \theta_3 \end{pmatrix} + 0.05 \times \varepsilon $$

where $\varepsilon$ is a 2-d standard Gaussian random variable.

In [5]:
def simulator(theta: torch.Tensor) -> torch.Tensor:
    x = torch.stack([
        theta[..., 0] + theta[..., 1] * theta[..., 2],
        theta[..., 0] * theta[..., 1] + theta[..., 2],
    ], dim=-1)

    return x + 0.05 * torch.randn_like(x)

theta = prior.sample()
x = simulator(theta)

print(theta, x, sep='\n')

tensor([ 0.9121, -0.5785,  0.7630])
tensor([0.4579, 0.3166])


This implementation is vectorized, which means that it can process a batch of parameters at once.

In [6]:
theta = prior.sample((4,))
x = simulator(theta)

print(theta, x, sep='\n')

tensor([[ 0.9290, -0.1757, -0.7820],
        [ 0.5494, -0.7134, -0.3816],
        [-0.2231, -0.6708,  0.0089],
        [-0.3692,  0.8460,  0.4389]])
tensor([[ 1.1476, -1.0494],
        [ 0.9043, -0.7716],
        [-0.3210,  0.2096],
        [-0.0251,  0.1417]])


## Sampling

Given a prior and a simulator, the [`JointLoader`](https://francois-rozet.github.io/lampe/api/data.html#lampe.data.JointLoader) class provided by the [`lampe.data`](https://francois-rozet.github.io/lampe/api/data.html) module creates an iterable [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) of batched parameters-observation pairs $(\theta, x) \sim p(\theta, x)$.

In [7]:
loader = lampe.data.JointLoader(prior, simulator, batch_size=256)

for theta, x in tqdm(islice(loader, 256), total=256):
    pass

100%|██████████| 256/256 [00:03<00:00, 84.28it/s]


If the simulator takes a NumPy array as input, it should be indicated with `numpy=True`. Additionally, if the simulator is vectorized, it is recommended to indicate it with `vectorized=True` as it can improve sampling performances significantly.

In [8]:
loader = lampe.data.JointLoader(prior, simulator, batch_size=256, vectorized=True)

for theta, x in tqdm(islice(loader, 256), total=256):
    pass

100%|██████████| 256/256 [00:00<00:00, 10390.68it/s]


## Saving on disk

If the simulator is fast or inexpensive, it is reasonable to generate pairs $(\theta, x)$ on demand. Otherwise, the pairs have to be generated and stored on disk ahead of time. The [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) file format is commonly used for this purpose, as it was specifically designed to hold large amounts of numerical data.

The [`lampe.data`](https://francois-rozet.github.io/lampe/api/data.html) module provides the [`H5Dataset`](https://francois-rozet.github.io/lampe/api/data.html#lampe.data.H5Dataset) class to help load and store pairs $(\theta, x)$ in HDF5 files. The [`H5Dataset.store`](https://francois-rozet.github.io/lampe/api/data.html#lampe.data.H5Dataset.store) function takes an iterable of batched pairs $(\theta, x)$ as input and stores them into a new HDF5 file. The iterable can be a precomputed list, a custom generator or even a `JointLoader` instance.

In [9]:
data = []

for _ in range(256):
    theta = prior.sample((256,))
    x = simulator(theta)
    
    data.append((theta, x))
    
lampe.data.H5Dataset.store(data, 'data_0.h5', size=2**16)

100%|██████████| 65536/65536 [00:00<00:00, 518158.51sample/s]


In [10]:
def generate():
    while True:
        theta = prior.sample((256,))
        x = simulator(theta)

        yield theta, x

lampe.data.H5Dataset.store(generate(), 'data_1.h5', size=2**16)

100%|██████████| 65536/65536 [00:00<00:00, 375452.15sample/s]


In [11]:
lampe.data.H5Dataset.store(loader, 'data_2.h5', size=2**16)

100%|██████████| 65536/65536 [00:00<00:00, 369042.89sample/s]


## Loading from disk

Now that the pairs are stored, we need to load them. The `H5Dataset` class creates an [`IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset) of pairs $(\theta, x)$ from HDF5 files. The pairs are dynamically loaded, meaning they are read from disk on demand instead of being cached in memory. This allows for very large datasets that do not even fit in memory.

In [12]:
dataset = lampe.data.H5Dataset('data_0.h5')

for i in tqdm(range(len(dataset))):
    theta, x = dataset[i]

100%|██████████| 65536/65536 [00:53<00:00, 1227.49it/s]


However, as it can be slow to read pairs from disk one by one, `H5Dataset` implements a custom `__iter__` method which loads pairs by chunks to reduce the number of disk reads.

In [13]:
for theta, x in tqdm(dataset):
    pass

100%|██████████| 65536/65536 [00:00<00:00, 188557.83it/s]


### Batching and shuffling

As `DataLoader` is not able to shuffle the elements of an `IterableDataset` instance, `H5Dataset` provides a `shuffle` argument to enable (disabled by default) shuffling the pairs when iterating. A `batch_size` argument is also available.

In [14]:
dataset = lampe.data.H5Dataset('data_0.h5', batch_size=4, shuffle=True)

In [15]:
for theta, x in dataset:
    print(theta, x, sep='\n')
    break

tensor([[-0.5102, -0.1597,  0.4225],
        [ 0.6263, -0.5490, -0.5424],
        [ 0.3914,  0.8860,  0.7201],
        [-0.6238,  0.9327, -0.6588]])
tensor([[-0.5276,  0.5434],
        [ 0.8239, -0.9316],
        [ 0.9018,  1.0367],
        [-1.3076, -1.2636]])


In [16]:
for theta, x in dataset:
    print(theta, x, sep='\n')
    break

tensor([[ 0.3951, -0.0103, -0.9181],
        [-0.5488, -0.9719, -0.5787],
        [-0.2422, -0.0027,  0.8907],
        [ 0.6688,  0.7993, -0.6854]])
tensor([[ 0.2745, -0.8634],
        [-0.0400, -0.0773],
        [-0.2196,  0.9108],
        [ 0.1359, -0.1790]])


## Merging datasets

Another feature of `H5Dataset` is that it can load data from any number of HDF5 files.

In [17]:
dataset = lampe.data.H5Dataset('data_0.h5', 'data_1.h5', 'data_2.h5')

for theta, x in tqdm(dataset):
    pass

100%|██████████| 196608/196608 [00:00<00:00, 206486.69it/s]


This feature is very useful when aggregating data from multiple sources, such as data generated on several machines or at different times.

In [18]:
dataset = lampe.data.H5Dataset('data_0.h5', 'data_1.h5', 'data_2.h5', batch_size=256)

lampe.data.H5Dataset.store(dataset, 'data_all.h5', size=len(dataset))

100%|██████████| 196608/196608 [00:00<00:00, 423420.86sample/s]


In [19]:
dataset = lampe.data.H5Dataset('data_all.h5')

for theta, x in tqdm(dataset):
    pass

100%|██████████| 196608/196608 [00:00<00:00, 226463.55it/s]
